In [1]:
import zipfile
from pathlib import Path
import os
import pandas as pd

# Extract the MRI for subjects with paired MRI and Protein with class labels

In [3]:
def extract_mprage_files(zip_path, target_folder):
    """
    Extract all MPRAGE files from zip, maintaining subject structure
    """
    import zipfile
    from pathlib import Path
    
    # Create target folder
    target_base = Path(target_folder)
    target_base.mkdir(exist_ok=True)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        all_items = zip_ref.namelist()
        
        # Find all MPRAGE-related items
        mprage_items = []
        for item in all_items:
            if 'MPRAGE' in item or 'MP-RAGE' in item:
                mprage_items.append(item)
        
        print(f"🔍 Found {len(mprage_items)} MPRAGE items")
        
        # Extract each MPRAGE item
        extracted_count = 0
        for item in mprage_items:
            try:
                # Parse the path: ADNI/133_S_0525/MPRAGE/...
                parts = item.split('/')
                
                # Find subject ID (e.g., "133_S_0525")
                subject_id = None
                for part in parts:
                    if '_S_' in part:
                        subject_id = part
                        break
                
                if subject_id:
                    # Create target path: target_folder/ADNI/subject_id/MPRAGE/...
                    # Remove the first part (zip root) and keep the rest
                    relative_path = '/'.join(parts[1:])  # Skip first empty part
                    target_path = target_base / relative_path
                    
                    # Create parent directories
                    target_path.parent.mkdir(parents=True, exist_ok=True)
                    
                    # Extract the file
                    with zip_ref.open(item) as source, open(target_path, 'wb') as target:
                        target.write(source.read())
                    
                    extracted_count += 1
                    if extracted_count <= 10:  # Show first 10
                        print(f"   📁 Extracted: {relative_path}")
                
            except Exception as e:
                print(f"❌ Error extracting {item}: {e}")
        
        print(f"✅ Successfully extracted {extracted_count} MPRAGE files")
        return extracted_count

# Usage
zip_path = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_5.zip"
target_folder = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_2"

print(f"🚀 Extracting MPRAGE files from: {zip_path}")
print(f"📁 Target folder: {target_folder}")

extracted_count = extract_mprage_files(zip_path, target_folder)

print(f"\n🎯 Extraction complete! Found {extracted_count} MPRAGE files")
print(f"�� Files extracted to: {target_folder}")

🚀 Extracting MPRAGE files from: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_5.zip
📁 Target folder: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_2
🔍 Found 208 MPRAGE items
   📁 Extracted: 002_S_0413/Accelerated_Sagittal_MPRAGE__MSV21_/2025-02-19_10_21_54.0/I11128519/ADNI_002_S_0413_MR_Accelerated_Sagittal_MPRAGE_(MSV21)__br_raw_20250226190620274_90.dcm
   📁 Extracted: 002_S_0413/Accelerated_Sagittal_MPRAGE__MSV21_/2025-02-19_10_21_54.0/I11128519/ADNI_002_S_0413_MR_Accelerated_Sagittal_MPRAGE_(MSV21)__br_raw_20250226190620365_179.dcm
   📁 Extracted: 002_S_0413/Accelerated_Sagittal_MPRAGE__MSV21_/2025-02-19_10_21_54.0/I11128519/ADNI_002_S_0413_MR_Accelerated_Sagittal_MPRAGE_(MSV21)__br_raw_20250226190620557_47.dcm
   📁 Extracted: 002_S_0413/Accelerated_Sagittal_MPRAGE__MSV21_/2025-02-19_10_21_54.0/I11128519/ADNI_002_S_0413_MR_Accelerated_Sagittal_MPRAGE_(MSV21)__br_raw_20250226190620686_101.dcm
   📁 Extracted: 002_S_0413/Acc

In [10]:
def count_subject_ids_at_level(zip_paths):
    """
    Count subject IDs at the specific directory level: ADNI/[Subject_ID]/
    """
    import zipfile
    from pathlib import Path
    from collections import defaultdict
    
    print("🔍 Counting subject IDs at ADNI/[Subject_ID] level across ZIP files...")
    print("=" * 70)
    
    # Dictionary to store subjects per ZIP
    zip_subjects = {}
    # Dictionary to track which subjects appear in multiple ZIPs
    subject_occurrences = defaultdict(list)
    
    for zip_path in zip_paths:
        zip_name = Path(zip_path).name
        print(f"\n📦 Processing: {zip_name}")
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                all_items = zip_ref.namelist()
                
                # Find subject IDs ONLY at the ADNI/[Subject_ID] level
                subjects_in_zip = set()
                for item in all_items:
                    parts = item.split('/')
                    
                    # Check if this item has the structure: ADNI/[Subject_ID]/...
                    if len(parts) >= 3:  # Must have at least 3 parts
                        if parts[0] == 'ADNI' and '_S_' in parts[1]:
                            # This is at the ADNI/[Subject_ID] level
                            subject_id = parts[1]
                            subjects_in_zip.add(subject_id)
                            subject_occurrences[subject_id].append(zip_name)
                
                zip_subjects[zip_name] = subjects_in_zip
                print(f"   ✅ Found {len(subjects_in_zip)} unique subjects at ADNI/[Subject_ID] level")
                
        except Exception as e:
            print(f"   ❌ Error processing {zip_name}: {e}")
            zip_subjects[zip_name] = set()
        
    return zip_subjects

# Define the 5 ZIP file paths
zip_paths = [
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_1 (1).zip",
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_2.zip",
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_3.zip",
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_4.zip",
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\MRI_5.zip"
]

# Count subject IDs at the specific level
print("🚀 Starting subject ID count at ADNI/[Subject_ID] level...")
zip_subjects= count_subject_ids_at_level(zip_paths)

🚀 Starting subject ID count at ADNI/[Subject_ID] level...
🔍 Counting subject IDs at ADNI/[Subject_ID] level across ZIP files...

📦 Processing: MRI_1 (1).zip
   ✅ Found 39 unique subjects at ADNI/[Subject_ID] level

📦 Processing: MRI_2.zip
   ✅ Found 39 unique subjects at ADNI/[Subject_ID] level

📦 Processing: MRI_3.zip
   ✅ Found 39 unique subjects at ADNI/[Subject_ID] level

📦 Processing: MRI_4.zip
   ✅ Found 39 unique subjects at ADNI/[Subject_ID] level

📦 Processing: MRI_5.zip
   ✅ Found 39 unique subjects at ADNI/[Subject_ID] level


# Extract Additional MRI Training Images

In [ ]:
# Return subject_id for 200 additional images for AD and CN

"D:\ADNI\AD_CN\AD,CN_My_Table_25Jun2025.csv"
# read csv into pandas df called ad_cn_df
# filter for visit is 'bl'

# read D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\rid_subject_mapping.csv into pandas
# call it as rid_subject_mapping_df

# in ad_cn_df drop the rows with subject_id in rid_subject_mapping_df

In [ ]:
ad_cn_df = pd.read_csv("D:\ADNI\AD_CN\AD,CN_My_Table_25Jun2025.csv")
print(f"Original AD/CN table shape: {ad_cn_df.shape}")

# Filter for baseline visit ('bl')
ad_cn_df = ad_cn_df[ad_cn_df['visit'] == 'bl']
print(f"After filtering for baseline visit: {ad_cn_df.shape}")

# Read the RID subject mapping file
rid_subject_mapping_df = pd.read_csv(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\rid_subject_mapping.csv")
print(f"RID subject mapping shape: {rid_subject_mapping_df.shape}")

# # Display sample of both DataFrames
# print(f"\n📋 Sample of AD/CN table:")
# print(ad_cn_df.head())
# print(f"\n📋 Sample of RID subject mapping:")
# print(rid_subject_mapping_df.head())

# Get the subject IDs from the mapping file
subject_ids_to_remove = rid_subject_mapping_df['subject_id'].unique()
print(f"\n�� Found {len(subject_ids_to_remove)} unique subject IDs to remove")

# Drop rows where subject_id exists in the mapping file
original_count = len(ad_cn_df)
ad_cn_df = ad_cn_df[~ad_cn_df['subject_id'].isin(subject_ids_to_remove)]
final_count = len(ad_cn_df)

print(f"\n✅ Filtering complete:")
print(f"   • Original rows: {original_count}")
print(f"   • Rows removed: {original_count - final_count}")
print(f"   • Final rows: {final_count}")

Original AD/CN table shape: (2693, 6)
After filtering for baseline visit: (804, 6)
RID subject mapping shape: (39, 2)

�� Found 39 unique subject IDs to remove

✅ Filtering complete:
   • Original rows: 804
   • Rows removed: 37
   • Final rows: 767


<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
C:\Users\User\AppData\Local\Temp\ipykernel_2820\1999474966.py:3: SyntaxWarning: invalid escape sequence '\A'
  ad_cn_df = pd.read_csv("D:\ADNI\AD_CN\AD,CN_My_Table_25Jun2025.csv")


In [9]:
print(ad_cn_df.head())

# Randomly sample 100 AD and 100 CN subjects
print(f"\n🎯 SAMPLING 100 AD AND 100 CN SUBJECTS")
print("=" * 50)

# Check current research_group distribution
print(f"Current research_group distribution:")
print(ad_cn_df['research_group'].value_counts())

# Randomly sample 100 AD subjects
ad_subjects = ad_cn_df[ad_cn_df['research_group'] == 'AD'].sample(n=100, random_state=42)
print(f"\n✅ Sampled {len(ad_subjects)} AD subjects")

# Randomly sample 100 CN subjects  
cn_subjects = ad_cn_df[ad_cn_df['research_group'] == 'CN'].sample(n=100, random_state=42)
print(f"✅ Sampled {len(cn_subjects)} CN subjects")

# Combine the sampled subjects
sampled_df = pd.concat([ad_subjects, cn_subjects], ignore_index=True)
print(f"✅ Combined sample size: {len(sampled_df)}")

# # Display the sampled subjects with subject_id and research_group
# print(f"\n📋 SAMPLED SUBJECTS:")
# print("=" * 60)
# print(f"{'Index':<6} {'subject_id':<15} {'research_group':<15}")
# print("-" * 60)

for idx, row in sampled_df.iterrows():
    print(f"{idx:<6} {row['subject_id']:<15} {row['research_group']:<15}")

# Print all 200 subject_ids with commas
print(f"\n📋 ALL 200 SUBJECT IDs (comma-separated):")
print("=" * 60)
subject_ids_list = sampled_df['subject_id'].tolist()
subject_ids_string = ','.join(map(str, subject_ids_list))
print(subject_ids_string)

# Show summary statistics
print(f"\n�� SAMPLE SUMMARY:")
print(f"   • Total subjects: {len(sampled_df)}")
print(f"   • AD subjects: {len(sampled_df[sampled_df['research_group'] == 'AD'])}")
print(f"   • CN subjects: {len(sampled_df[sampled_df['research_group'] == 'CN'])}")

# Verify the sampling worked correctly
print(f"\n🔍 VERIFICATION:")
print(f"   • AD count = 100: {'✅' if len(ad_subjects) == 100 else '❌'}")
print(f"   • CN count = 100: {'✅' if len(cn_subjects) == 100 else '❌'}")
print(f"   • Total = 200: {'✅' if len(sampled_df) == 200 else '❌'}")

# Save the sampled data if needed
# sampled_df.to_csv('sampled_ad_cn_subjects.csv', index=False)
# print(f"\n�� Sampled data saved to 'sampled_ad_cn_subjects.csv'")

   subject_id visit  PROTEIN research_group subject_date  subject_age
1  011_S_0023    bl     30.0             CN   2005-10-18        71.79
2  011_S_0010    bl     35.0             AD   2005-10-26        73.90
3  011_S_0005    bl     43.0             CN   2005-08-23        73.73
4  011_S_0003    bl     86.0             AD   2005-08-18        81.30
5  011_S_0008    bl     41.0             CN   2005-08-29        84.50

🎯 SAMPLING 100 AD AND 100 CN SUBJECTS
Current research_group distribution:
research_group
CN    516
AD    251
Name: count, dtype: int64

✅ Sampled 100 AD subjects
✅ Sampled 100 CN subjects
✅ Combined sample size: 200
0      033_S_5017      AD             
1      023_S_0083      AD             
2      127_S_5028      AD             
3      130_S_1337      AD             
4      135_S_4657      AD             
5      005_S_5119      AD             
6      073_S_5090      AD             
7      033_S_10249     AD             
8      005_S_0221      AD             
9      137_

# Extract additional MRI training data

In [ ]:
import zipfile
import os
import re
from pathlib import Path
from tqdm import tqdm

def extract_relevant_files_simple(zip_path, extract_to):
    """
    Extract relevant MRI scan files from zip archive based on regex pattern matching.
    
    This function performs intelligent filtering to extract only specific MRI scan types
    (MPRAGE, IR-SPGR, FSPGR, BRAVO, 3D T1, SAG T1) from a large zip archive while 
    maintaining the original directory structure.
    
    Args:
        zip_path (str): Path to the zip file containing MRI data
        extract_to (str): Directory to extract files to
    
    Returns:
        int: Number of files successfully extracted
        
    Features:
        - Filters by scan type using regex pattern matching
        - Maintains original folder structure: ADNI/subject_id/scan_type/date/ImageData/
        - Shows progress bar during extraction
        - Handles extraction errors gracefully
        - Provides detailed logging of the process
    """
    # Validate zip file exists and is readable
    zip_path = Path(zip_path)
    if not zip_path.exists():
        print(f"❌ Error: Zip file does not exist: {zip_path}")
        return 0
    
    if not zip_path.is_file():
        print(f"❌ Error: Path is not a file: {zip_path}")
        return 0
    
    # Check file size
    file_size = zip_path.stat().st_size
    print(f"📊 Zip file size: {file_size:,} bytes ({file_size / (1024**3):.2f} GB)")
    
    # Create extract directory if it doesn't exist
    extract_to = Path(extract_to)
    extract_to.mkdir(exist_ok=True)
    
    # Regex pattern to match relevant scan types
    scan_pattern = re.compile(r'(mp-?rage|mprage|ir-?spgr|fspgr|bravo|3d\s*t1|sag.*t1)', re.IGNORECASE)
    
    print(f"📦 Extracting relevant files from: {zip_path.name}")
    print(f"📁 To: {extract_to}")
    print(f"🔍 Filtering for scan types: MPRAGE, IR-SPGR, FSPGR, BRAVO, 3D T1, SAG T1")
    
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Test if zip file is readable
            try:
                all_files = zip_ref.namelist()
                print(f"📊 Total files in zip: {len(all_files)}")
            except Exception as e:
                print(f"❌ Error reading zip file contents: {e}")
                return 0
            
            # Filter files that match our scan types
            relevant_files = []
            for file_path in all_files:
                # Check if path contains .dcm files and matches scan pattern
                if file_path.endswith('.dcm') and scan_pattern.search(file_path):
                    relevant_files.append(file_path)
            
            print(f"🎯 Found {len(relevant_files)} relevant files to extract")
            
            if not relevant_files:
                print("❌ No relevant files found!")
                return 0
            
            # Extract relevant files with progress bar
            extracted_count = 0
            with tqdm(total=len(relevant_files), desc="Extracting", unit="files", 
                     bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} files [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
                for file_path in relevant_files:
                    try:
                        zip_ref.extract(file_path, extract_to)
                        extracted_count += 1
                        pbar.update(1)
                    except Exception as e:
                        print(f"❌ Error extracting {file_path}: {e}")
                        pbar.update(1)
            
            print(f"✅ Extraction complete!")
            print(f"📁 Files extracted to: {extract_to}")
            print(f"📊 Successfully extracted {extracted_count} relevant files out of {len(all_files)} total files")
            
            return extracted_count
            
    except zipfile.BadZipFile as e:
        print(f"❌ Error: The file is not a valid zip file or is corrupted: {e}")
        print(f"💡 Try checking if the file downloaded completely or re-download it")
        return 0
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return 0

# Check if the zip file exists and get the correct path
zip_file_candidates = [
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data.zip"
]

print("🔍 Checking for zip files...")
valid_zip = None
for zip_path in zip_file_candidates:
    if Path(zip_path).exists():
        print(f"✅ Found: {zip_path}")
        valid_zip = zip_path
        break
    else:
        print(f"❌ Not found: {zip_path}")

if valid_zip:
    extract_directory = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images"
    print(f"\n🚀 Using zip file: {valid_zip}")
    extracted_count = extract_relevant_files_simple(valid_zip, extract_directory)
else:
    print("❌ No valid zip file found. Please check the file paths.")


🔍 Checking for zip files...
✅ Found: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data.zip

🚀 Using zip file: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data.zip
📊 Zip file size: 143,511,955,991 bytes (133.66 GB)
📦 Extracting relevant files from: AD_CN_all_available_data.zip
📁 To: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images
🔍 Filtering for scan types: MPRAGE, IR-SPGR, FSPGR, BRAVO, 3D T1, SAG T1
📊 Total files in zip: 2253646
🎯 Found 1703742 relevant files to extract


Extracting:  44%|████▍     | 746689/1703742 files [12:05:15<25:10:21, 10.56files/s] 

❌ Error extracting ADNI/126_S_0605/IR-FSPGR__replaces_MP-Rage_/2011-07-05_07_56_13.0/I244002/ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162133248_135_S114135_I244002.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0605\\IR-FSPGR__replaces_MP-Rage_\\2011-07-05_07_56_13.0\\I244002\\ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162133248_135_S114135_I244002.dcm'
❌ Error extracting ADNI/126_S_0605/IR-FSPGR__replaces_MP-Rage_/2011-07-05_07_56_13.0/I244002/ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162139607_110_S114135_I244002.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0605\\IR-FSPGR__replaces_MP-Rage_\\2011-07-05_07_56_13.0\\I244002\\ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162139607_110_S114135_I244002

Extracting:  44%|████▍     | 746703/1703742 files [12:05:15<12:02:06, 22.09files/s]

❌ Error extracting ADNI/126_S_0605/IR-FSPGR__replaces_MP-Rage_/2011-07-05_07_56_13.0/I244002/ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162138591_105_S114135_I244002.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0605\\IR-FSPGR__replaces_MP-Rage_\\2011-07-05_07_56_13.0\\I244002\\ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162138591_105_S114135_I244002.dcm'
❌ Error extracting ADNI/126_S_0605/IR-FSPGR__replaces_MP-Rage_/2011-07-05_07_56_13.0/I244002/ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162143810_111_S114135_I244002.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0605\\IR-FSPGR__replaces_MP-Rage_\\2011-07-05_07_56_13.0\\I244002\\ADNI_126_S_0605_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302162143810_111_S114135_I244002

...

Extracting:  44%|████▍     | 755507/1703742 files [12:13:20<7:47:42, 33.79files/s] 

❌ Error extracting ADNI/126_S_0680/IR-FSPGR__replaces_MP-Rage_/2011-08-10_15_00_44.0/I250391/ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160322123_165_S118613_I250391.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0680\\IR-FSPGR__replaces_MP-Rage_\\2011-08-10_15_00_44.0\\I250391\\ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160322123_165_S118613_I250391.dcm'
❌ Error extracting ADNI/126_S_0680/IR-FSPGR__replaces_MP-Rage_/2011-08-10_15_00_44.0/I250391/ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160309498_163_S118613_I250391.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0680\\IR-FSPGR__replaces_MP-Rage_\\2011-08-10_15_00_44.0\\I250391\\ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160309498_163_S118613_I250391

Extracting:  44%|████▍     | 755515/1703742 files [12:13:20<7:48:24, 33.74files/s]

❌ Error extracting ADNI/126_S_0680/IR-FSPGR__replaces_MP-Rage_/2011-08-10_15_00_44.0/I250391/ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160323295_129_S118613_I250391.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0680\\IR-FSPGR__replaces_MP-Rage_\\2011-08-10_15_00_44.0\\I250391\\ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160323295_129_S118613_I250391.dcm'
❌ Error extracting ADNI/126_S_0680/IR-FSPGR__replaces_MP-Rage_/2011-08-10_15_00_44.0/I250391/ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160323326_164_S118613_I250391.dcm: [Errno 2] No such file or directory: 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\126_S_0680\\IR-FSPGR__replaces_MP-Rage_\\2011-08-10_15_00_44.0\\I250391\\ADNI_126_S_0680_MR_IR-FSPGR__replaces_MP-Rage___br_raw_20120302160323326_164_S118613_I250391

Extracting:  47%|████▋     | 796596/1703742 files [12:50:39<15:26:17, 16.32files/s]

# Util


In [45]:
from pathlib import Path

def count_next_level_directories(path):
    """
    Count the number of directories at the next level of the given path.
    
    Args:
        path (str): The directory path to check
        
    Returns:
        int: Number of directories at the next level
        
    Example:
        path = "D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images"
        # This will count directories inside the 'extracted_images' folder
    """
    path = Path(path)
    
    # Check if path exists
    if not path.exists():
        print(f"❌ Path does not exist: {path}")
        return 0
    
    # Check if path is a directory
    if not path.is_dir():
        print(f"❌ Path is not a directory: {path}")
        return 0
    
    # Get all items in the directory
    try:
        items = list(path.iterdir())
    except PermissionError:
        print(f"❌ Permission denied accessing: {path}")
        return 0
    except Exception as e:
        print(f"❌ Error accessing directory: {e}")
        return 0
    
    # Count only directories
    directories = [item for item in items if item.is_dir()]
    directory_count = len(directories)
    
    print(f"📁 Path: {path}")
    print(f"📊 Found {directory_count} directories at the next level")
    
    if directory_count > 0:
        print(f"📋 Directory names:")
        for i, dir_path in enumerate(directories, 1):
            print(f"   {i}. {dir_path.name}")
    else:
        print("   (No directories found)")
    
    return directory_count

# Test the function with your example path
test_path = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI"
count = count_next_level_directories(test_path)

📁 Path: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI
📊 Found 1715 directories at the next level
📋 Directory names:
   1. 011_S_0002
   2. 011_S_0003
   3. 011_S_0005
   4. 022_S_0007
   5. 011_S_0008
   6. 100_S_0015
   7. 011_S_0016
   8. 022_S_0014
   9. 067_S_0019
   10. 011_S_0021
   11. 011_S_0022
   12. 067_S_0020
   13. 023_S_0031
   14. 067_S_0029
   15. 067_S_0024
   16. 100_S_0035
   17. 099_S_0040
   18. 011_S_0023
   19. 011_S_0010
   20. 067_S_0056
   21. 011_S_0053
   22. 100_S_0047
   23. 035_S_0048
   24. 022_S_0066
   25. 023_S_0061
   26. 023_S_0058
   27. 010_S_0067
   28. 007_S_0068
   29. 018_S_0055
   30. 067_S_0076
   31. 018_S_0043
   32. 067_S_0059
   33. 100_S_0069
   34. 023_S_0083
   35. 027_S_0074
   36. 123_S_0091
   37. 123_S_0094
   38. 032_S_0095
   39. 023_S_0093
   40. 023_S_0084
   41. 068_S_0109
   42. 123_S_0072
   43. 123_S_0088
   44. 023_S_0081
   45. 099_S_0090
   46. 020_S_0097
  

In [31]:
import pandas as pd

def count_unique_subjects(csv_path):
    """
    Count the number of unique subjects in the CSV file.
    
    Args:
        csv_path (str): Path to the CSV file
        
    Returns:
        int: Number of unique subjects
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        print(f"📊 CSV file loaded successfully")
        print(f"📋 Total rows: {len(df)}")
        print(f"📋 Total columns: {len(df.columns)}")
        
        # Check if 'Subject' column exists
        if 'Subject' not in df.columns:
            print(f"❌ 'Subject' column not found in CSV")
            print(f"📋 Available columns: {list(df.columns)}")
            return 0
        
        # Count unique subjects
        unique_subjects = df['Subject'].nunique()
        total_subjects = len(df['Subject'])
        
        print(f"🎯 Unique subjects: {unique_subjects}")
        print(f"📊 Total subject entries: {total_subjects}")
        
        # Show some sample subjects
        sample_subjects = df['Subject'].unique()[:10]
        print(f"📋 Sample subjects (first 10):")
        for i, subject in enumerate(sample_subjects, 1):
            print(f"   {i}. {subject}")
        
        if unique_subjects > 10:
            print(f"   ... and {unique_subjects - 10} more")
        
        return unique_subjects
        
    except FileNotFoundError:
        print(f"❌ File not found: {csv_path}")
        return 0
    except Exception as e:
        print(f"❌ Error reading CSV: {e}")
        return 0

# Check unique subjects in the CSV file
csv_path = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data.csv"
unique_count = count_unique_subjects(csv_path)

📊 CSV file loaded successfully
📋 Total rows: 31182
📋 Total columns: 12
🎯 Unique subjects: 1719
📊 Total subject entries: 31182
📋 Sample subjects (first 10):
   1. 941_S_7087
   2. 941_S_7074
   3. 941_S_7051
   4. 941_S_7046
   5. 941_S_6998
   6. 941_S_6962
   7. 941_S_6854
   8. 941_S_6607
   9. 941_S_6581
   10. 941_S_6580
   ... and 1709 more


In [32]:
## This means there are 1719 subjects in the csv file with AD or CN labels

In [35]:
from pathlib import Path

dirs = [
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI",
    r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images\ADNI",
    # r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_91gb",
]

def subdir_names(p: str) -> set[str]:
    p = Path(p)
    return {d.name for d in p.iterdir() if d.is_dir()} if p.exists() else set()

all_unique = set().union(*(subdir_names(d) for d in dirs))
print(f"Unique subject folders across all 3: {len(all_unique)}")

## This means we have 1717 AD or CN subjects

Unique subject folders across all 3: 1717


In [38]:
from pathlib import Path

# Input roots (subject_id folders are the next level; inside them are scan_type folders)
dir_a = Path(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images\ADNI")
dir_b = Path(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI")

def subject_to_scan_types(root: Path) -> dict[str, set[str]]:
    """
    Build a mapping: subject_id -> set of scan_type folder names under that subject.
    Only directories are considered; files are ignored.
    """
    mapping: dict[str, set[str]] = {}
    if not root.exists():
        print(f"❌ Missing directory: {root}")
        return mapping

    for subject_dir in root.iterdir():
        if not subject_dir.is_dir():
            continue
        subj = subject_dir.name
        # Collect immediate subdirectories as scan types
        scan_types = {
            p.name for p in subject_dir.iterdir() if p.is_dir()
        }
        mapping[subj] = scan_types
    return mapping

# Build mappings for both trees
map_a = subject_to_scan_types(dir_a)
map_b = subject_to_scan_types(dir_b)

# Subject presence
subjects_a = set(map_a.keys())
subjects_b = set(map_b.keys())
common_subjects = subjects_a & subjects_b
only_in_a = subjects_a - subjects_b
only_in_b = subjects_b - subjects_a

print(f"Total subjects in A: {len(subjects_a)}")
print(f"Total subjects in B: {len(subjects_b)}")
print(f"Common subjects in both: {len(common_subjects)}")
print(f"Only in A: {len(only_in_a)}")
print(f"Only in B: {len(only_in_b)}")

# Compare scan_type sets for common subjects
mismatched = []
for subj in sorted(common_subjects):
    scans_a = map_a[subj]
    scans_b = map_b[subj]
    if scans_a != scans_b:
        # Compute differences for clarity
        only_scans_in_a = sorted(scans_a - scans_b)
        only_scans_in_b = sorted(scans_b - scans_a)
        mismatched.append((subj, only_scans_in_a, only_scans_in_b))

print(f"\nSubjects with identical scan_type sets: {len(common_subjects) - len(mismatched)}")
print(f"Subjects with differing scan_type sets: {len(mismatched)}")

# # Optionally print a few detailed mismatches for inspection
# max_show = 20  # adjust as needed
# for subj, only_scans_in_a, only_scans_in_b in mismatched[:max_show]:
#     print(f"\nSubject: {subj}")
#     if only_scans_in_a:
#         print(f"  In A only ({len(only_scans_in_a)}): {only_scans_in_a}")
#     if only_scans_in_b:
#         print(f"  In B only ({len(only_scans_in_b)}): {only_scans_in_b}")

# If you want a complete list saved to a file, uncomment below:
from csv import writer
out_csv = Path("D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\scan_type_mismatches.csv")
with out_csv.open("w", newline="", encoding="utf-8") as f:
    w = writer(f)
    w.writerow(["subject_id", "in_A_only", "in_B_only"])
    for subj, a_only, b_only in mismatched:
        w.writerow([subj, ";".join(a_only), ";".join(b_only)])
print(f"\nSaved detailed mismatches to: {out_csv.resolve()}")

<>:70: SyntaxWarning: invalid escape sequence '\A'
<>:70: SyntaxWarning: invalid escape sequence '\A'
C:\Users\User\AppData\Local\Temp\ipykernel_22000\3740308785.py:70: SyntaxWarning: invalid escape sequence '\A'
  out_csv = Path("D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\scan_type_mismatches.csv")


Total subjects in A: 537
Total subjects in B: 1715
Common subjects in both: 535
Only in A: 2
Only in B: 1180

Subjects with identical scan_type sets: 398
Subjects with differing scan_type sets: 137

Saved detailed mismatches to: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\scan_type_mismatches.csv


In [44]:
## COPY only missing stuff in A to B 
import csv
from pathlib import Path
import shutil

# Paths
csv_path = Path(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\scan_type_mismatches.csv")
A_root = Path(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images\ADNI")
B_root = Path(r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI")

def parse_scan_list(s: str) -> list[str]:
    if not s:
        return []
    return [t.strip() for t in s.split(';') if t.strip()]

def main():
    if not csv_path.exists():
        print(f"❌ CSV not found: {csv_path}")
        return
    A_root.mkdir(parents=True, exist_ok=True)
    B_root.mkdir(parents=True, exist_ok=True)

    total_rows = 0
    rows_with_A_only = 0
    total_A_only_scans = 0

    copied = 0
    skipped_exists = 0
    missing_source = 0
    errors = 0

    with csv_path.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            total_rows += 1
            subj = row.get("subject_id", "").strip()
            a_only_field = row.get("in_A_only", "")

            scans = parse_scan_list(a_only_field)
            if not scans:
                continue

            rows_with_A_only += 1
            total_A_only_scans += len(scans)

            for scan in scans:
                src = A_root / subj / scan
                dst = B_root / subj / scan

                if not src.exists():
                    missing_source += 1
                    print(f"❌ Missing source: {src}")
                    continue

                if dst.exists():
                    skipped_exists += 1
                    print(f"⏭ Already exists, skipping: {dst}")
                    continue

                try:
                    dst.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copytree(src, dst)  # copy whole scan-type folder
                    copied += 1
                    print(f"✅ Copied: {src} -> {dst}")
                except Exception as e:
                    errors += 1
                    print(f"❌ Error copying {src} -> {dst}: {e}")

    print("\nSummary")
    print(f"CSV rows processed: {total_rows}")
    print(f"Rows with in_A_only: {rows_with_A_only}")
    print(f"Total scan-type folders to copy (in_A_only): {total_A_only_scans}")
    print(f"Copied: {copied}")
    print(f"Skipped (dest exists): {skipped_exists}")
    print(f"Missing sources: {missing_source}")
    print(f"Errors: {errors}")

if __name__ == "__main__":
    main()

✅ Copied: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images\ADNI\010_S_0472\MPRAGE -> D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI\010_S_0472\MPRAGE
❌ Error copying D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\extracted_images\ADNI\012_S_1009\ADNI_______MPRAGEadni2B -> D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\MRI\AD_CN_all_available_data_total\ADNI\012_S_1009\ADNI_______MPRAGEadni2B: [('D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\extracted_images\\ADNI\\012_S_1009\\ADNI_______MPRAGEadni2B\\2006-11-03_14_02_32.0\\I28961\\ADNI_012_S_1009_MR_ADNI_______MPRAGEadni2B__br_raw_20061107101329828_86_S22178_I28961.dcm', 'D:\\ADNI\\AD_CN\\proteomics\\Biomarkers Consortium Plasma Proteomics MRM\\MRI\\AD_CN_all_available_data_total\\ADNI\\012_S_1009\\ADNI_______MPRAGEadni2B\\2006-11-03_14_02_32.0\\I28961\\A